In [22]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
import random
import re


In [2]:
X_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_x_resample3000_boundarysel_50pergroup.csv")
y_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_y_resample3000_boundarysel_50pergroup.csv")
X_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\scaled_validation_feature.csv")
y_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\validation_label.csv")

In [3]:
X_train = X_train.drop(["predict_cluster"],axis=1)
y_train = y_train.drop(["predict_cluster"],axis=1)
X_val = X_val.drop(["Unnamed: 0"],axis=1)
y_val = y_val.drop(["Unnamed: 0"],axis=1)

In [5]:
dataset = loadmat ("D:\\lab; signal processing\\forStudents\\medData\\dataset.mat",mat_dtype=True)

In [34]:
data=dataset["dataset"]

In [35]:
def cal_score (y_pred,y_val):
    n11 = 0
    n12 = 0
    n21 = 0
    n22 = 0
    y_pred_array= np.array(y_pred)
    y_val_array= np.array(y_val)
    for j in range(len(y_pred_array)):
        if (y_pred_array[j]==2)&(y_val_array[j]==2):
            n22 = n22+1
        elif (y_pred_array[j]==1)&(y_val_array[j]==2):
            n12 = n12 +1
        elif (y_pred_array[j]==2)&(y_val_array[j]==1):
            n21 = n21+1
        else:
            n11 = n11+1   
    '''
    ita = np.nditer (y_pred_array)
    itb = np.nditer (y_val_array)
    while not ita.finished:
        if (ita[0]==2)&(itb[0]==2):
            n22 = n22+1
        elif (ita[0]==1)&(itb[0]==2):
            n12 = n12+1
        elif (ita[0]==2)&(itb[0]==1):
            n21 = n21+1
        else:
            n11 = n11+1
        ita.iternext()
        itb.iternext()
    '''
    try:       
        Precall = n22 / ( n12 + n22)
        Pprecision = n22 / ( n21 + n22)
        f1_score = 2 / (1/Precall + 1/Pprecision)
        FP = n21/(n21 + n11)
        FN = n12/(n12 +n22 )
        BER = 1/2*(FP+FN)   
    except Exception as ex:
        print (ex.message)
    print ("n11:.."+str(n11)+"..n12:.."+str(n12)+"..n21:.."+str(n21)+"..n22:.."+str(n22))
    print ("TP:"+str(Precall))
    print ("f1 score:" + str(f1_score))
    print ("FP:"+ str(FP))
    print ("BER:" + str(BER))
    return Precall,f1_score,BER,FP

In [29]:
def predicate_of_doctor (data,y_val):
    doctorA = []
    doctorB = []
    for i in range(len(y_val)):
        temparr = re.findall("\d+",y_val.iloc[i]['indexofpixel'])
        a=int(temparr[0])
        b=int(temparr[1])
        c=int(temparr[2])
        d=int(y_val.iloc[i]['sourceofpixel'])
        doctorA.append(data[d][0][0][0][1][a][b][c])
        doctorB.append(data[d][0][0][0][2][a][b][c])
    return doctorA,doctorB

In [30]:
doctorA,doctorB=predicate_of_doctor(data,y_val)

In [36]:
doctorA = {"label_pred":doctorA}
doctorA = pd.DataFrame(data=doctorA)
doctorB = {"label_pred":doctorB}
doctorB = pd.DataFrame(data=doctorB)

In [39]:
cal_score(doctorA,y_val['label'])

n11:..53283..n12:..2647..n21:..7924..n22:..6173
TP:0.6998866213151927
f1 score:0.538726709429681
FP:0.12946231640171876
BER:0.21478784754326302


(0.6998866213151927,
 0.538726709429681,
 0.21478784754326302,
 0.12946231640171876)

In [40]:
cal_score(doctorB,y_val['label'])

n11:..51963..n12:..2334..n21:..9244..n22:..6486
TP:0.7353741496598639
f1 score:0.5283910386965376
FP:0.15102847713496823
BER:0.20782716373755217


(0.7353741496598639,
 0.5283910386965376,
 0.20782716373755217,
 0.15102847713496823)